***STEP 1:IMPORTS***

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [0]:
# step 1 load the data 
from tensorflow.examples.tutorials.mnist import input_data

data = input_data.read_data_sets("data/MNIST/",one_hot=True)


# sub step is used to get the no. form 0 - 9 form the labels

#data.train.cls = [ labes.argmax() for labels in data.train.labels]

data.test.cls = np.argmax(data.test.labels,axis=1)


#we need to flatten the image so 28 * 28

img_size = 28

img_size_flat = 28*28

num_channels = 1 # no. of color channels for img. 1 for gray-scale

# no. of classes

num_classes =10 # 10 for 10 digits


Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


**STEP 2:CREATE MODEL**

In [0]:
#Setp 2 :Creating the model

#Sub step a: setting up variables for the cnn

#Convolution Layer 1;

filter_size1 = 5 #convolution filters 5 x 5 pixels (image size 748)
num_filters1 = 16 # There are 16 of these filters

#Convolution Layer 2.
filter_size2 = 5
num_filters2 = 36 

#Fully - connected layer (sometimes called the dence layer )

fc_size = 128 # no of nerons in fully connected layer.

# creatig the model
def new_weights(shape):
  return tf.Variable(tf.truncated_normal(shape,stddev=0.05))

def new_biases(length):
  return tf.Variable(tf.constant(0.05,shape=[length]))

'''input is 4 dim tensor 
   1.image no 
   2. y-axis  of each image
   3.x-axis of each image
   4.channels of each image
'''

''' output is 4 dim tensor
    1.img no. same as input
    2.y-axis of each img . if pooling is used 2 x 2 (height x width)/2
    3.x-axis
    4.Channel
'''



#function for creating the convolutin layer
def new_conv_layer(input1, #the output of previous layer 
                   num_input_channels,#Num.of channels in prev. layer
                   filter_size ,#Width and height of each filter
                   num_filters,
                   use_pooling=True):
  #use 2x2 max-pooling
  
  #shape of filter weights for convolution
  
  shape = [filter_size,filter_size,num_input_channels,num_filters]
  
  #create a new weight , filters with given shape.
  
  weights = new_weights(shape=shape)
  
  #create new  biases
  
  biases = new_biases(length=num_filters)
  
  

  '''note learn more abt. strides'''
  
  #create the first layer
  
  layer = tf.nn.conv2d(input=input1,filter = weights,strides=[1,1,1,1],padding='SAME')
  
  
  
  #Add the biases to the result of the convolution.
  #A bias-value is added to each filter-channel.  
  
  layer+=biases
  
  '''addition step over here is the pooling layer'''
  
  if use_pooling:
    
    layer = tf.nn.max_pool(value=layer,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    
  
  #activation layer using relu
  '''working : it calculates max(x,0) for each input'''
   
  layer = tf.nn.relu(layer)  

  return layer,weights





#function for flattening layer 
''' input is 4 dim tensor  output is 2 dim tensor'''

def flatten_layer(layer):
  
  # get the shape of the input layer
  
  layer_shape = layer.get_shape()
  
  #the no. of features is:img_height*img_width,num_channel
  new_features = layer_shape[1:4].num_elements()
  '''find out more on the tensorflow operation'''
  
  layer_flat = tf.reshape(layer,[-1,new_features])
  
  return layer_flat,new_features


#function for fully-connected layer


def new_fc_layer(input,num_inputs,num_outputs,use_relu=True):
  
  weights = new_weights(shape=[num_inputs,num_outputs])
  
  biases =  new_biases(length=num_outputs)
  
  layer = tf.matmul(input,weights)+biases
  
  
  
  #use relu
  
  if use_relu:
     layer = tf.nn.relu(layer)
      
  return layer    
  





**Placeholders**

In [0]:
x = tf.placeholder(shape=[None,784],dtype=tf.float32 , name="x") # used for taken in the images

# since convolution layer excepts a 4 dim tensor
# num_images x image_height x img_width x no_of_channels
x_image = tf.reshape(x,[-1,img_size,img_size,num_channels])

#y _true will give a matrx containing labels 
y_true = tf.placeholder(shape=[None,num_classes],dtype=tf.float32,name="y")

#get the no. form the labels

y_true_cls = tf.argmax(y_true,axis=1)



**CREATING AND CONNECTION LAYERS

In [0]:
#layer 1
layer_conv1, weights_conv1 = new_conv_layer(input1=x_image,num_input_channels=num_channels,filter_size=filter_size1,num_filters=num_filters1,use_pooling=True)

layer_conv1

#layer 2


layer_conv2, weights_conv2 = \
    new_conv_layer(input1=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)



layer_conv2

#layer 3 flatten layer

layer_flat,num_features = flatten_layer(layer_conv2)

layer_flat

num_features


#layer 4 fully connected layer

layer_fc1 = new_fc_layer(input=layer_flat,num_inputs=num_features,num_outputs=fc_size,use_relu=False)

layer_fc1

#layer 5 fully connected layer 2

layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False)

layer_fc2



<tf.Tensor 'add_24:0' shape=(?, 10) dtype=float32>

In [0]:
y_pred = tf.nn.softmax(layer_fc2)

y_pred_cls = tf.argmax(y_pred,axis=1)

**STEP 3: CLACULATE COST**

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,labels=y_true))

**STEP 4: INITIALIZE THE OPTIMIZER**

In [0]:
optimizer1 = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [0]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

In [0]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**STEP 5:RUN FUNCTON**

In [0]:
session  = tf.Session()
session.run(tf.global_variables_initializer())
train_batch_size = 100


In [0]:
def optimizer(num_iterations):
  
  for i in range(num_iterations):
    
    x_batch,y_batch = data.train.next_batch(train_batch_size)
    
    
    #y_true_cls is only used  during testing
    
    feed_dict_train = {x:x_batch,y_true:y_batch}
    
    session.run(optimizer1,feed_dict=feed_dict_train)
   
    if i % 100 == 0:
      
      acc = session.run(accuracy,feed_dict={x:data.train.images,y_true:data.train.labels})
      print("accuracy:.2%".format(acc))
      
    
  
  
  
  
  
  

In [0]:
optimize(100)

Optimization Iteration:      1, Training Accuracy:  13.0%


**STEP 6 : ADDITIONAL FUNCTONS FOR  PLOTTING IMAGE GRAPHS**

In [0]:
# helper function for potting images
def plot_images(images, cls_true, cls_pred=None):
    #assert len(images) == len(cls_true) == 9
    
    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Plot image.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
      
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [0]:
#function to display error images
def plot_example_errors(cls_pred, correct):
    # This function is called from print_test_accuracy() below.

    # cls_pred is an array of the predicted class-number for
    # all images in the test-set.

    # correct is a boolean array whether the predicted class
    # is equal to the true class for each image in the test-set.

    # Negate the boolean array.
    incorrect = (correct == False)
    
    # Get the images from the test-set that have been
    # incorrectly classified.
    images = data.test.images[incorrect]
    
    # Get the predicted classes for those images.
    cls_pred = cls_pred[incorrect]

    # Get the true classes for those images.
    cls_true = data.test.cls[incorrect]
    
    # Plot the first 9 images.
    plot_images(images=images[0:9],
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])

In [0]:
def print_accuracy():
  
  
  feed_dict_test = {x:data.test.images,y_true:data.test.labels}
  acc = session.run(accuracy,feed_dict=feed_dict_test)
  
  print("Accuracy fo test-set:{0:.4%}".format(acc))

In [0]:
print_accuracy()
optimizer(100)

Accuracy o test-set:95.1400%
